# Serial Optimization
How do you optimise serial code? We cover some of the techniques that you might use for serial optimization in Python: 
1. Vectorization
2. Memoization
3. Compilation

## Vectorization
Because Python is an interpreted language - i.e. each line is dispatched to an interpreter program to interpret and execute - there can be a lot of overhead compared to compiled programs. Python needs to check variable types and use the correct functions for the inputs, because these are not necessarily specified. As a result, when you're using Python for large datasets or for long loops, it can pay to implement vectorization.

Vectorization essentially increases memory locality, and allows the CPU to use SIMD instructions.  Single Instruction, Multiple Data (SIMD) instructions - these allow the same instruction to be performed on multiple cells of data simultaneously. There is therefore much less overhead per operation, providing large speedups. Improving memory locality also reduces the overhead of dereferencing lots of values that are in lots of different places in memory - contiguous memory blocks that contain all the data you need are loaded on to the CPU instead.

Consider the following examples. The first uses base Python to generate a list of integers using a for loop. The second does the same, but uses `numpy` to vectorize the calculation. The `%%timeit` readouts show a large speed increase in the second example.

In [45]:
def double_ints():
  result = []
  for x in range(1_000_000):
    result.append(x * 2)
  return result


In [46]:
%timeit double_ints()

36.2 ms ± 711 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
import numpy as np

In [47]:
def np_double_ints():
  ints = np.arange(1_000_000)
  return ints * 2

In [48]:
assert double_ints() == np_double_ints().tolist(), "Results do not match!"

In [49]:
%timeit np_double_ints()

1.72 ms ± 28.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


We see a large speed increase in the second example, which uses `numpy` to vectorize the calculation.

### Vectorization Exercises
The follow code calculates $\pi$ using Monte Carlo integration. Answer the questions.

The following code, which calculates pi using Monte Carlo integration, is provided. Answer the following questions:
1. Is it correct?
2. How long does it take to run?
3. Where is the most time taken, cumulatively?

Then:
1. Vectorize the code
2. Verify the outputs
3. Measure any speedup.

In [2]:
import random

def monte_carlo_pi(num_samples):
    inside_circle = 0
    for _ in range(num_samples):
        x, y = random.uniform(-1, 1), random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1
    return (inside_circle / num_samples) * 4

**Q1.1: Is `monte_carlo_pi()` correct? At what number of samples are you accurate to three digits?**

<details>
<summary>I need a hint</summary>
Run the function with varying `num_samples` and decide whether, as `num_samples` increases, the output approaches 3.1415
</details>

**Q1.2: How long does the function take to run, on average, when `num_samples = 1_000_000`? How does this scale as you vary the number of samples**

<details>
<summary>I need a hint</summary>

Use the `time` module or use `%timeit`
<details>
<summary>I need a bigger hint</summary>

Write a wrapper function which runs and times the function.
<details>
<summary>Give me the code</summary>

```{Python}
def pi_time(n):
    import time
    start = time.time()
    out = monte_carlo_pi(n)
    end = time.time()
    return (out, end - start)
```
</details>
</details>
</details>



**Q1.3: What functions are called most in `monte_carlo_pi()`?**

<details>
<summary>I need a hint</summary>

Use `cProfile`
<details>
<summary>I need a bigger hint</summary>

Either use `%prun` magic, or call `cProfile()` directly.
<details>
<summary>Give me the code</summary>

```{Python}
%prun monte_carlo_pi(10_000_000)
```

OR

```{Python}
import cProfile
cProfile.run("monte_carlo_pi(1_000_000)")
```
</details>
</details>
</details>

**Q1.4: Where is the most time taken in `monte_carlo_pi()`, cumulatively?**

<details>
<summary>I need a hint</summary>

You want to know which line takes most time.
<details>
<summary>I need a bigger hint</summary>

Use `line_profiler` or `%lprun`.
<details>
<summary>Give me the code</summary>

```{Python}
%load_ext line_profiler
%lprun -f monte_carlo_pi monte_carlo_pi(10_000_000)
```
</details>
</details>
</details>


In [13]:
%load_ext line_profiler
%lprun -f monte_carlo_pi monte_carlo_pi(1_000_000)

Timer unit: 1e-09 s

Total time: 1.17552 s
File: /tmp/ipykernel_2855/3916854312.py
Function: monte_carlo_pi at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def monte_carlo_pi(num_samples):
     4         1        600.0    600.0      0.0      inside_circle = 0
     5   1000001  108869600.0    108.9      9.3      for _ in range(num_samples):
     6   1000000  700355101.0    700.4     59.6          x, y = random.uniform(-1, 1), random.uniform(-1, 1)
     7   1000000  261651600.0    261.7     22.3          if x**2 + y**2 <= 1:
     8    784967  104646900.0    133.3      8.9              inside_circle += 1
     9         1        800.0    800.0      0.0      return (inside_circle / num_samples) * 4


**Q1.5: Can you speed up the function?**

<details>
<summary>I need a hint</summary>

Previous results should have shown most of the time is taken generating random numbers, and checking whether the random point is inside a circle. Can you vectorise either of these?
<details>
<summary>I need a bigger hint</summary>

`numpy` has functions that vectorise random number generation, and allows you to vectorise exponentiation.
<details>
<summary>Give me the code</summary>

```{Python}
def monte_carlo_pi_numpy(num_samples):
  x, y = np.random.uniform(-1, 1, (2, num_samples)) # vectorised random number generation
  pi = 4 * np.mean(x**2 + y**2 <= 1) # perform check simultaneously on all datav`\` has functions that vectorise random number generation, as wend allows you to vectorise exponentiation.pnumpy
  return pi
```
</details>
</details>
</details>


**Q1.6: Is your edited function correct? Is it faster? Are there any other ways you can think of to speed it up?**

## Memoization
Memoization is a technique to cache results when they're calculated so they can be quickly retrieved when the same input is called. It avoids expensive computation of the same results again and again.

Let's remind ourselves of the function we defined previously to calculate the $n^{th}$ number of the Fibonacci sequence.

In [20]:
def fib(n):
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)

In [ ]:
%timeit fib(35)

80.9 ms ± 450 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


On our computer, `fib(35)` takes 892 ms on average.

We can implement a cache simply ourselves, using dictionaries.

In [30]:
cache = {}
def fib_cache(n):
    if n in cache:
        return cache[n]
    if n < 2:
        result = n
    else:
        result = fib(n-1) + fib(n-2)
    cache[n] = result
    return result


In [ ]:
assert fib(35) == fib_cache(35), "Cached result does not match non-cached result!"
%timeit fib_cache(35)

60.7 ns ± 0.626 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


On our computer, calling `fib_cache(35)` took an average of 61 nanoseconds. Merely implementing a basic cache resulted in approximately a 1-million-fold speedup.

Due to the recursive nature of this function, the speedup achieved will be greater a n increases, due to the greater number of avoided computations as n grows.

We can also use the `functools` module, which comes with Python and provides the function decorator `@lru_cache`. LRU stands for Least Recently Used - when the cache hits its size limit it removes the least recently used data to make room for the new data. 

In [36]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib_lru(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [37]:
%timeit fib_lru(35)

257 ns ± 469 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


On our computer, the cached version took 257 nanoseconds, *slower* than using our dictionary. Does that hold on your computer? Did you expect it? The functools version avoids code complexity and makes it very simple to cache, however, in this case it does not appear to be as fast as using a dictionary - reiterating the importance of measuring all changes to your code.


### Memoization Exercises

We provide a function below that computes the Binomial Coefficient ("n choose k") recursively.

In [54]:
def binomial(n, k):
    if k == 0 or k == n:
        return 1
    return binomial(n - 1, k - 1) + binomial(n - 1, k)


**Q2.1: How long does this function take with n = 20, k = 10? Explore how the time taken changes as a function of n or k?**

<details>
<summary>I need a hint</summary>
Time the code with a variety of n/k.
<details>
<summary>I need a bigger hint</summary>
Use %timeit, or write a wrapper function.
<details>
<summary>Give me the code</summary>

Edit the code below and explore.
```
%timeit binomial(20, 10)
```
</details>
</details>
</details>


**Q2.2: How many recursive function calls are there with n = 20, k = 10?**

<details>
<summary>I need a hint</summary>

Use `cProfile`.
<details>
<summary>I need a bigger hint</summary>

Either use `cProfile.run()` or `%prun`.
<details>
<summary>Give me the code</summary>

```{Python}
%prun binomial(20, 10)
```
</details>
</details>
</details>


**Q2.3 Implement memoization. How fast is the function with the same inputs you tested before? How many function calls are there?**

<details>
<summary>I need a hint</summary>

Cache the results if not already in the cache.
<details>
<summary>I need a bigger hint</summary>

Either use a dictionary cache, or `@lru_cache` from `functools`, then time it and use `cProfile`.
<details>
<summary>Give me the code</summary>

```{Python}
@lru_cache(maxsize=None)
def binomial(n, k):
    if k == 0 or k == n:
        return 1
    return binomial(n - 1, k - 1) + binomial(n - 1, k)

%timeit binomial(20, 10)

%prun binomial(20, 10)
```
</details>
</details>
</details>

## Compilation of functions
Python, as an interpreted language, has many overheads as we've discussed. To gain code flexibility and speed of implementation, the language had to make many dynamic checks as code is interpreted. The `CPython` interpreter is well implemented, and these checks are individually fairly fast, but over millions of computations the overhead adds up. In addition, there are implementation details that can simply make speedy code impossible - such as Python lists being arrays of pointers meaning every loop over them involves fetching data twice (get the pointer, then get the data where the pointer points to).

There are, however, ways of stepping around the interpreter, and compiling our code ourselves for use in Python. The longest running and most mature method of this is `Cython`, which involves learning a small amount of `C` and setting up a toolchain. We will not be covering that, but if you're interested I encourage you to check it out at https://cython.org/.

We will cover the `Numba` package. `Cython` is an Ahead-Of-Time (AOT) compiler - you compile your function(s) before using, creating a static library that can instantly be used. In comparison, `Numba` is a Just-In-Time (JIT) compiler - when your function is first called it will compile the function then, and *future* uses of that function will benefit from the compiled version. The first use, then, will be slow due to the compilation, with all subsequent calls being (hopefully) much faster.

If you're using `Numba`, you simply add a decorator and away we go. There are two decorators to consider, because `Numba` has a `nopython` mode. In `nopython` mode the function is compiled to machine code without using the Python interpreter *at all* - however, if this is not possible it fails. Avoiding all the interpreter overhead is often worth it, though. 

- `@jit()` - tells `Numba` that you want this function to be compiled. It will try to use `nopython`, but if it fails it will fall back to using the python interpreter. Use `@jit(nopython=True)` to enforce `nopython` mode.
- `@njit()` - tell `Numba` to compile the function in `nopython` mode.

So, lets start using `Numba`. Below we code a Sieve of Eratosthenes using `numpy` to find all primes up to a number `n`. For a primer on the Sieve of Eratosthenes, read the [wikipedia page](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)

In [ ]:
def sieve_primes(n):
    is_prime = np.ones(n + 1, dtype=np.uint8) # Create a boolean array
    is_prime[:2] = 0  # 0 and 1 are not prime numbers
    for i in range(2, int(n**0.5) + 1):
        if is_prime[i] == 1:
            is_prime[i*i:n+1:i] = 0 # Mark multiples of i as non-prime
    return np.where(is_prime)[0] # Get indices of True values, which are the prime numbers

assert len(sieve_primes(10000)) == 1229, "sieve_primes did not return expected number of primes!"

%timeit sieve_primes(100000)



290 μs ± 8.74 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


On our computer, the sieve of eratosthenes took 161 microseconds on average, when n = 100,000. 

Adding the `@jit` decorator:

In [82]:
from numba import njit

@njit()
def sieve_primes_numba(n):
    is_prime = np.ones(n + 1, dtype=np.uint8) # Create a boolean array
    is_prime[:2] = 0  # 0 and 1 are not prime numbers
    for i in range(2, int(n**0.5) + 1):
        if is_prime[i] == 1:
            is_prime[i*i:n+1:i] = 0 # Mark multiples of i as non-prime
    return np.where(is_prime)[0] # Get indices of True values, which are the prime numbers


In [84]:
assert len(sieve_primes_numba(10000)) == 1229, "sieve_primes did not return expected number of primes!"

%timeit sieve_primes_numba(100000) 

196 μs ± 981 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Compilation exercises